In [ ]:
import os
import glob
import json
import warnings

# Data science / numeric
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew

# scikit-learn
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from scipy.stats import kstest

# Probabilistic scoring
from properscoring import crps_gaussian, crps_ensemble

# PyTorch (keep if you use it later)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Utilities
import joblib
import optuna

# Suppress warnings in notebook output
warnings.filterwarnings("ignore")

# 0) Importing Data and Preprocessing 

In [ ]:
#############################################################################
# Import data
#############################################################################
x_test = pd.read_csv('X_test.csv')
x_trn = pd.read_csv('X_trn.csv')
y_trn = pd.read_csv('y_trn.csv')

xy_trn = x_trn.copy()
xy_trn.insert(0, 'realrinc', y_trn.squeeze())
xy_trn = xy_trn.sort_values(by='year').reset_index(drop=True) # sorted by year

x_test_transformed = x_test.copy() # will transform below 

#############################################################################
# Preprocessing TRAIN DATA 
#############################################################################
df = xy_trn.copy()

### TARGET variable: "realrinc" ###
# Apply log(1 + y) to reduce skewness
# Then standardize
df["realrinc"] = np.log1p(df["realrinc"])
mean_y = df["realrinc"].mean()
std_y  = df["realrinc"].std()
df["realrinc"] = (df["realrinc"] - mean_y) / std_y

### REMAINING features ###
## AGE
mean_age = df["age"].mean()
std_age  = df["age"].std()
df["age"] = (df["age"] - mean_age) / std_age

## pretg10
mean_prestg10 = df["prestg10"].mean()
std_prestg10  = df["prestg10"].std()
df["prestg10"] = (df["prestg10"] - mean_prestg10) / std_prestg10

## Childs
# Apply log(1 + x) 
# Then standardize
df["childs"] = np.log1p(df["childs"])
mean_childs = df["childs"].mean()
std_childs  = df["childs"].std()
df["childs"] = (df["childs"] - mean_childs) / std_childs

## Gender 
df["gender"] = np.where(df["gender"] == "Male", 1, 0)

## Marital Status (one-hot encoding)
df = pd.get_dummies(df, columns=["maritalcat"], drop_first=False)
marital_cols = [col for col in df.columns if col.startswith("maritalcat_")]
df[marital_cols] = df[marital_cols].astype(int)

## Work Status (one-hot encoding)
df = pd.get_dummies(df, columns=["wrkstat"], drop_first=False)
wrkstat_cols = [col for col in df.columns if col.startswith("wrkstat_")]
df[wrkstat_cols] = df[wrkstat_cols].astype(int)

## Occupation (one-hot encoding)
df = pd.get_dummies(df, columns=["occrecode"], drop_first=False)
occrecode_cols = [col for col in df.columns if col.startswith("occrecode_")]
df[occrecode_cols] = df[occrecode_cols].astype(int)

## Education (ordinal encodigng)
edu_map = {
    "Less Than High School": 0,
    "High School": 1,
    "Junior College": 2,
    "Bachelor": 3,
    "Graduate": 4
}
df["educcat"] = df["educcat"].map(edu_map)

### STORE: KEEP TRACK FOR LATER ###
scalers = {
    "realrinc": {"mean": mean_y, "std": std_y, "log_transform": True},
    "age": {"mean": mean_age, "std": std_age, "log_transform": False},
    "prestg10": {"mean": mean_prestg10, "std": std_prestg10, "log_transform": False},
    "childs": {"mean": mean_childs, "std": std_childs, "log_transform": True}
}

# Separating X/y
y_trn_new = df["realrinc"].copy()
x_trn_new = df.drop(columns=["realrinc"]).copy()

#############################################################################
# Preprocessing TEST DATA
#############################################################################
df = x_test.copy()

### FEATURES ###

## AGE — use training mean/std
df["age"] = (df["age"] - scalers["age"]["mean"]) / scalers["age"]["std"]

## prestg10 — use training mean/std
df["prestg10"] = (df["prestg10"] - scalers["prestg10"]["mean"]) / scalers["prestg10"]["std"]

## Childs — log + standardize using training mean/std
df["childs"] = np.log1p(df["childs"])
df["childs"] = (df["childs"] - scalers["childs"]["mean"]) / scalers["childs"]["std"]

## Gender — same binary encoding as train
df["gender"] = np.where(df["gender"] == "Male", 1, 0)

## Marital Status (one-hot encoding)
df = pd.get_dummies(df, columns=["maritalcat"], drop_first=False)
marital_cols = [col for col in df.columns if col.startswith("maritalcat_")]
df[marital_cols] = df[marital_cols].astype(int)

## Work Status (one-hot encoding)
df = pd.get_dummies(df, columns=["wrkstat"], drop_first=False)
wrkstat_cols = [col for col in df.columns if col.startswith("wrkstat_")]
df[wrkstat_cols] = df[wrkstat_cols].astype(int)

## Occupation (one-hot encoding)
df = pd.get_dummies(df, columns=["occrecode"], drop_first=False)
occrecode_cols = [col for col in df.columns if col.startswith("occrecode_")]
df[occrecode_cols] = df[occrecode_cols].astype(int)

## Education (ordinal encoding)
edu_map = {
    "Less Than High School": 0,
    "High School": 1,
    "Junior College": 2,
    "Bachelor": 3,
    "Graduate": 4
}
df["educcat"] = df["educcat"].map(edu_map)

# Align columns with training data 
df = df.reindex(columns=x_trn_new.columns, fill_value=0)

# Copy the result to x_test_transformed
x_test_transformed = df.copy()

# 1) Define the Model (QRF) and helper functions that are used throughout

In [ ]:
def _weighted_quantile(x, q, w):
    x = np.asarray(x); w = np.asarray(w)
    s = np.argsort(x); x, w = x[s], w[s]
    cw = np.cumsum(w) / np.sum(w)
    return np.interp(q, cw, x)

class QRF:
    """
    Quantile Regression Forest using leaf-weight aggregation over training targets.
    - predict_quantiles(): weighted quantiles of training y
    - sample(): draws from empirical conditional distribution using leaf weights
    """
    def __init__(self, **rf_params):
        self.model = RandomForestRegressor(**rf_params)
        self.leaf_maps_ = None
        self.y_train_ = None
        self.n_train_ = None

    def fit(self, X, y):
        X = pd.DataFrame(X).reset_index(drop=True)
        y = pd.Series(y).reset_index(drop=True).values
        self.model.fit(X, y)
        self.y_train_ = y
        self.n_train_ = len(y)

        # For each tree: map leaf_id -> indices of training samples in that leaf
        self.leaf_maps_ = []
        for tree in self.model.estimators_:
            leaves = tree.apply(X)
            m = {}
            for lid in np.unique(leaves):
                m[lid] = np.where(leaves == lid)[0]
            self.leaf_maps_.append(m)
        return self

    def _weights_for(self, Xq):
        Xq = pd.DataFrame(Xq)
        n_q = len(Xq)
        W = np.zeros((n_q, self.n_train_), dtype=float)
        n_trees = len(self.model.estimators_)
        for t, tree in enumerate(self.model.estimators_):
            lids = tree.apply(Xq)
            mp = self.leaf_maps_[t]
            for i, lid in enumerate(lids):
                idx = mp[lid]
                W[i, idx] += 1.0 / (n_trees * len(idx))
        # each row sums to 1
        return W

    def predict_quantiles(self, X, quantiles):
        qs = np.asarray(quantiles).ravel()
        W = self._weights_for(X)
        out = np.empty((len(X), len(qs)), dtype=float)
        for i in range(len(X)):
            out[i] = [_weighted_quantile(self.y_train_, q, W[i]) for q in qs]
        return out

    def sample(self, X, n_samples=1000, rng=None):
        if rng is None:
            rng = np.random.default_rng(1738)
        W = self._weights_for(X)
        draws = np.empty((len(X), n_samples), dtype=float)
        for i in range(len(X)):
            draws[i] = rng.choice(self.y_train_, size=n_samples, p=W[i])
        return draws

def crps_from_samples(y_true, samples):
    y_true = np.asarray(y_true).reshape(-1, 1)
    samples = np.asarray(samples)
    n, S = samples.shape
    A = np.mean(np.abs(samples - y_true), axis=1)
    xs = np.sort(samples, axis=1)
    k = np.arange(1, S + 1, dtype=float)
    weights = 2 * k - S - 1
    B = (2.0 / (S**2)) * np.sum(xs * weights, axis=1)
    return A - 0.5 * B

def _predict_quantiles_any(model, X, alphas):
    for attr in ["predict_quantiles", "predict", "quantile", "predict_quantile"]:
        if hasattr(model, attr):
            try:
                q = getattr(model, attr)(X, alphas)
            except TypeError:
                q = getattr(model, attr)(X, q=alphas)
            q = np.asarray(q)
            if q.ndim == 1 and len(alphas) == 1:
                q = q.reshape(-1, 1)
            if q.shape[1] == len(alphas):
                return q
    raise NotImplementedError("No quantile method found.")

def inverse_cdf_sample_from_quantiles(qvals, alphas, n_samples, rng):
    qvals = np.asarray(qvals)
    alphas = np.asarray(alphas).ravel()
    n, m = qvals.shape
    U = rng.uniform(alphas[0], alphas[-1], size=(n, n_samples))
    out = np.empty((n, n_samples), dtype=float)
    for i in range(n):
        out[i] = np.interp(U[i], alphas, qvals[i])
    return out

def get_predictive_samples(model, X, n_samples=200, rng=None, alphas=None):
    if rng is None:
        rng = np.random.default_rng(1738)
    if alphas is None:
        alphas = np.linspace(0.01, 0.99, 199)
    if hasattr(model, "sample"):
        try:
            draws = np.asarray(model.sample(X, n_samples=n_samples))
            if draws.shape == (len(X), n_samples):
                return draws
        except Exception:
            pass
    qvals = _predict_quantiles_any(model, X, alphas)
    return inverse_cdf_sample_from_quantiles(qvals, alphas, n_samples, rng)

def cv_crps_for_params(X, y, qrf_cls, params, K=5, S=200, seed=1738):
    if not isinstance(X, (pd.DataFrame, pd.Series)):
        X = pd.DataFrame(X)
    y = pd.Series(y).reset_index(drop=True)
    kf = KFold(n_splits=K, shuffle=True, random_state=seed)
    rng = np.random.default_rng(seed)
    scores = []
    for tr, va in kf.split(X):
        Xtr, Xva = X.iloc[tr], X.iloc[va]
        ytr, yva = y.iloc[tr], y.iloc[va]
        model = qrf_cls(**params)
        model.fit(Xtr, ytr)
        samples = get_predictive_samples(model, Xva, n_samples=S, rng=rng)
        scores.append(np.mean(crps_from_samples(yva.values, samples)))
    return float(np.mean(scores))

def sample_qrf_params(rng):
    n_estimators = int(rng.choice([50, 200, 400, 800])) 
    max_depth = int(rng.integers(4, 20))            
    min_samples_leaf = int(rng.integers(2, 10))     
    mf_choice = rng.choice(['sqrt', 'log2'])
    return dict(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        max_features=mf_choice,
        bootstrap=True,
        random_state=1738,
        n_jobs=-1
    )

def random_search_qrf(X, y, qrf_cls, n_trials=60, K=5, S=200, seed=1738, verbose=False):
    rng = np.random.default_rng(seed)
    recs, best = [], {"score": np.inf, "params": None}
    for t in range(1, n_trials + 1):
        params = sample_qrf_params(rng)
        score = cv_crps_for_params(X, y, qrf_cls, params, K=K, S=S, seed=seed)
        recs.append({**params, "cv_crps": score})
        if score < best["score"]:
            best = {"score": score, "params": params}
        if verbose:
            print(f"[{t}/{n_trials}] CV-CRPS={score:.6f} | best={best['score']:.6f}")
    df = pd.DataFrame.from_records(recs).sort_values("cv_crps")
    return df, best

def fit_best_and_write_predictions(X, y, X_test, qrf_cls, best_params, n_samples=1000, seed=1738):
    if not isinstance(X, (pd.DataFrame, pd.Series)):
        X = pd.DataFrame(X)
    if not isinstance(X_test, (pd.DataFrame, pd.Series)):
        X_test = pd.DataFrame(X_test)
    model = qrf_cls(**best_params)
    model.fit(X, y)
    rng = np.random.default_rng(seed)
    draws = get_predictive_samples(model, X_test, n_samples=n_samples, rng=rng)
    return model, draws

def inverse_transform(y_scaled, mean_y, std_y, log_transform=True):
    y_unscaled = y_scaled * std_y + mean_y
    return np.expm1(y_unscaled) if log_transform else y_unscaled

def compute_pit(y_true, samples):
    """
    Compute Probability Integral Transform (PIT) values
    given samples from predictive distribution.
    """
    # For each observation, compute fraction of samples <= actual value
    return np.mean(samples <= y_true[:, None], axis=1)

def gini(x):
    x = np.sort(np.asarray(x))
    n = len(x)
    cumx = np.cumsum(x)
    return (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n

def sample_income_qrf(year, gender, n_samples=1000, rng=None):
    """
    Draw income samples from QRF model for a given year and gender.
    - year: numeric (e.g., 2000)
    - gender: 0 for Female, 1 for Male
    """
    Xq = pd.DataFrame({
        "year": [year],   
        "gender": [gender]  
    })
    log_draws = qrf.sample(Xq, n_samples=n_samples, rng=rng).ravel()
    return np.expm1(log_draws)  # Back-transform from log income

def prob_male_greater(year, n=1000, rng=None):
    m = sample_income_qrf(year, 1, n_samples=n, rng=rng)
    f = sample_income_qrf(year, 0, n_samples=n, rng=rng)
    return float(np.mean(m > f)), m, f

# 2) Model training/hyperparameter opt via k-fold CV
- 2.1) Random Search for HOP via 5-fold CV
- 2.2) Train model on a 80/20 split to compute the CRPS value 
- 2.3) Train model on all of the data and get predictions for X_test


In [ ]:
# 2.1) Hyperparameter tuning
# FOR QRF
X = x_trn_new.copy()
y = y_trn_new.copy()

results_df, best = random_search_qrf(
    X, y, QRF, n_trials=20, K=5, S=200, seed=1738, verbose=True
)

# We store the best hyperparameters found by the random search
# Uncomment to use best params from random search
#best_params = best["params"] 

In [ ]:
# Best hyperparameters found (tuned manually after random search)
best_params = {
    'n_estimators': 800,
    'max_depth': 18,
    'min_samples_leaf': 4,
    'max_features': "sqrt",
    'bootstrap': True,
    'random_state': 1738,  # synced with your random search seed
    'n_jobs': -1
}


In [ ]:
# 2.2) CRPS for 1 single validation set + PIT Plot
X = x_trn_new.copy()
y = y_trn_new.copy()

# Split original training set into new training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=1738
)

# Fit model on 80% of original training set
final_model_crps = QRF(**best_params).fit(X_train, y_train)

# Prediction samples for 20% validation set
samples_z = get_predictive_samples(final_model_crps, X_val, n_samples=1000)

# --- Inverse transform both samples and targets ---
mean_y = scalers["realrinc"]["mean"]
std_y  = scalers["realrinc"]["std"]
log_transform = scalers["realrinc"]["log_transform"]

y_val_orig = inverse_transform(y_val, mean_y, std_y, log_transform)
samples_orig = inverse_transform(samples_z, mean_y, std_y, log_transform)

# --- Compute CRPS in $ ---
crps_values = crps_from_samples(y_val_orig, samples_orig)
mean_crps_dollars = np.mean(crps_values)
print(f"Mean CRPS on original scale: {mean_crps_dollars:,.2f} USD")

#######################################################################################
# PIT PLOT
#######################################################################################

# Assuming y_val_orig and samples_orig are in the ORIGINAL scale
pit_values = compute_pit(np.asarray(y_val_orig), np.asarray(samples_orig))

# KS STAT
ks_stat, ks_p = kstest(pit_values, 'uniform')
print(f"KS statistic = {ks_stat:.3f}, p-value = {ks_p:.3f}")

# Plot PIT histogram
plt.figure(figsize=(6,4))
plt.hist(pit_values, bins=20, range=(0,1), edgecolor="black", density=True)
plt.axhline(1, color="red", linestyle="--", label="Uniform(0,1)")
plt.title("PIT Histogram – QRF model")
plt.xlabel("PIT value")
plt.ylabel("Density")
plt.legend()
plt.show()

In [ ]:
# 2.3) Train model on all of the data + get predictions for X_test
model, draws_scaled = fit_best_and_write_predictions(
    X, y, x_test_transformed,
    qrf_cls=QRF,
    best_params=best_params,
    n_samples=1000,
    seed=1738
)

mean_y = scalers["realrinc"]["mean"]
std_y  = scalers["realrinc"]["std"]
log_transform = scalers["realrinc"]["log_transform"]

# Revert back to original Scale for predictions
draws_unscaled = inverse_transform(
    draws_scaled,
    mean_y=scalers["realrinc"]["mean"],
    std_y=scalers["realrinc"]["std"],
    log_transform=scalers["realrinc"]["log_transform"]
)

np.save("predictions.npy", draws_unscaled)
print("Saved predictions.npy with shape:", draws_unscaled.shape)

# 3) Gender Subquestions

In [ ]:
# 3.1) Train Model
# Load data to ensure correctness
x_trn = pd.read_csv("X_trn.csv")
y_trn = pd.read_csv("y_trn.csv").squeeze()

data = x_trn[["year", "gender"]].copy()
data["gender"] = np.where(data["gender"] == "Male", 1, 0)
data["log_income"] = np.log1p(y_trn)

X = data[["year", "gender"]]
y = data["log_income"]

# We fit a QRF with sensible defaults given only 2 features 
# and lack of time to tune
qrf = QRF(
    n_estimators=800,
    max_depth=20,
    min_samples_leaf=5,
    max_features="sqrt",
    bootstrap=True,
    random_state=42,
    n_jobs=-1
).fit(X, y)


In [ ]:
 # 3.2) Subquestion 1
rng = np.random.default_rng(1738)
male_1980   = sample_income_qrf(1980, 1, 1000, rng=rng)
female_1980 = sample_income_qrf(1980, 0, 1000, rng=rng)

res_1980 = {
    "Male_STD":   float(np.std(male_1980, ddof=1)),
    "Female_STD": float(np.std(female_1980, ddof=1)),
    "Male_Gini":  gini(male_1980),
    "Female_Gini":gini(female_1980),
}

# PRINT RESULTS 
print("\n=== Income Inequality (1980, QRF) ===")
print(f"Male   → Std: {res_1980['Male_STD']:.2f},  Gini: {res_1980['Male_Gini']:.3f}")
print(f"Female → Std: {res_1980['Female_STD']:.2f},  Gini: {res_1980['Female_Gini']:.3f}")

# Histogram 
plt.figure(figsize=(4,3))
plt.hist(male_1980, bins=30, alpha=0.6, density=True, label="Male", color="steelblue", edgecolor="black")
plt.hist(female_1980, bins=30, alpha=0.6, density=True, label="Female", color="orange", edgecolor="black")

plt.title("Income Distribution by Gender (1980)", fontsize=9)
plt.xlabel("Real Income (USD)", fontsize=8)
plt.ylabel("Density", fontsize=8)
plt.xticks(fontsize=7)
plt.yticks(fontsize=7)
plt.legend(fontsize=7)
plt.tight_layout()
plt.show()

In [ ]:
# 3.3) Subquestion 2
rng = np.random.default_rng(42)

# 1980 samples 
male_1980   = sample_income_qrf(1980, 1, n_samples=1000, rng=rng)
female_1980 = sample_income_qrf(1980, 0, n_samples=1000, rng=rng)

diff_1980 = male_1980 - female_1980
prob_male_gt_female_1980 = np.mean(diff_1980 > 0)

# 2010 samples
male_2010   = sample_income_qrf(2010, 1, n_samples=1000, rng=rng)
female_2010 = sample_income_qrf(2010, 0, n_samples=1000, rng=rng)

diff_2010 = male_2010 - female_2010
prob_male_gt_female_2010 = np.mean(diff_2010 > 0)

# PRINT RESULTS 
print("\n=== Probability that Male Earned More than Female ===")
print(f"1980 → P(Male > Female): {prob_male_gt_female_1980:.3f}")
print(f"2010 → P(Male > Female): {prob_male_gt_female_2010:.3f}")

# Plot histogram for 1980 
plt.figure(figsize=(4,3))
plt.hist(diff_1980, bins=30, color="steelblue", edgecolor="black", density=True)
plt.axvline(0, color="red", linestyle="--", label="Equal income")
plt.title("Predicted Income Differences (1980)", fontsize=9)
plt.xlabel("Income difference (USD)", fontsize=8)
plt.ylabel("Density", fontsize=8)
plt.xticks(fontsize=7)
plt.yticks(fontsize=7)
plt.legend(fontsize=7)
plt.tight_layout()
plt.show()

# Plot histogram for 2010
plt.figure(figsize=(4,3))
plt.hist(diff_2010, bins=30, color="orange", edgecolor="black", density=True)
plt.axvline(0, color="red", linestyle="--", label="Equal income")
plt.title("Predicted Income Differences (2010)", fontsize=9)
plt.xlabel("Income difference (USD)", fontsize=8)
plt.ylabel("Density", fontsize=8)
plt.xticks(fontsize=7)
plt.yticks(fontsize=7)
plt.legend(fontsize=7)
plt.tight_layout()
plt.show()

In [ ]:
# Plot male vs female for 1980
plt.figure(figsize=(4,3))
plt.hist(male_1980 / 1000, bins=30, alpha=0.6, label="Male",
         color="steelblue", edgecolor="black", density=True)
plt.hist(female_1980 / 1000, bins=30, alpha=0.6, label="Female",
         color="orange", edgecolor="black", density=True)
plt.title("Predicted Income Distributions (1980)", fontsize=9)
plt.xlabel("Income (thousands of USD)", fontsize=8)
plt.ylabel("Density", fontsize=8)
plt.legend(fontsize=7)
plt.tight_layout()
plt.show()

# Plot male vs female for 2010
plt.figure(figsize=(4,3))
plt.hist(male_2010 / 1000, bins=30, alpha=0.6, label="Male",
         color="steelblue", edgecolor="black", density=True)
plt.hist(female_2010 / 1000, bins=30, alpha=0.6, label="Female",
         color="orange", edgecolor="black", density=True)
plt.title("Predicted Income Distributions (2010)", fontsize=9)
plt.xlabel("Income (thousands of USD)", fontsize=8)
plt.ylabel("Density", fontsize=8)
plt.legend(fontsize=7)
plt.tight_layout()
plt.show()
